# Test the pathline code

In [ ]:
import numpy as np
import pylab as plt
import xarray as xr
from pismragis.trajectories import compute_trajectory, trajectories_to_geopandas
from shapely import Point
from osgeo import ogr, osr
from joblib import Parallel, delayed
import geopandas as gp
from typing import Union
from pismragis.processing import tqdm_joblib
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
def create_circular() -> xr.Dataset:
    """
    Create xr.Dataset with radial velocity field
    """
    time = pd.date_range("2000-01-01", periods=1)
    reference_time = pd.Timestamp("2000-01-01")

    nx = 201
    ny = 401
    x = np.linspace(-100e3, 100e3, nx)
    y = np.linspace(-100e3, 100e3, ny)
    X, Y = np.meshgrid(x, y)

    # Directional vectors
    vx = -Y / np.sqrt(X**2 + Y**2) * 250
    vy = X / np.sqrt(X**2 + Y**2) * 250
    v = np.sqrt(vx**2 + vy**2)

    vx = vx.reshape(1, ny, nx)
    vy = vy.reshape(1, ny, nx)
    v = v.reshape(1, ny, nx)

    v_err = v / 10
    vx_err = np.abs(vx / 20)
    vy_err = np.abs(vy / 20)

    coords = {
        "x": (
            ["x"],
            x,
            {
                "units": "m",
                "axis": "X",
                "standard_name": "projection_x_coordinate",
                "long_name": "x-coordinate in projected coordinate system",
            },
        ),
        "y": (
            ["y"],
            y,
            {
                "units": "m",
                "axis": "Y",
                "standard_name": "projection_y_coordinate",
                "long_name": "y-coordinate in projected coordinate system",
            },
        ),
        "time": (["time"], time, {}),
    }

    ds = xr.Dataset(
        {
            "vx": xr.DataArray(
                data=vx,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in x-direction", "units": "m/yr"},
            ),
            "vy": xr.DataArray(
                data=vy,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in y-direction", "units": "m/yr"},
            ),
            "v": xr.DataArray(
                data=v,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={
                    "standard_name": "magnitude",
                    "units": "m/yr",
                    "grid_mapping": "polar_stereographic",
                },
            ),
            "vx_err": xr.DataArray(
                data=vx_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in x-direction", "units": "m/yr"},
            ),
            "vy_err": xr.DataArray(
                data=vy_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in y-direction", "units": "m/yr"},
            ),
            "v_err": xr.DataArray(
                data=v_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={
                    "standard_name": "magnitude",
                    "units": "m/yr",
                    "grid_mapping": "polar_stereographic",
                },
            ),
        },
        attrs={"Conventions": "CF-1.7"},
    )
    ds["Polar_Stereographic"] = int()
    ds.Polar_Stereographic.attrs["grid_mapping_name"] = "polar_stereographic"
    ds.Polar_Stereographic.attrs["false_easting"] = 0.0
    ds.Polar_Stereographic.attrs["false_northing"] = 0.0
    ds.Polar_Stereographic.attrs["latitude_of_projection_origin"] = 90.0
    ds.Polar_Stereographic.attrs["scale_factor_at_projection_origin"] = 1.0
    ds.Polar_Stereographic.attrs["standard_parallel"] = 70.0
    ds.Polar_Stereographic.attrs["straight_vertical_longitude_from_pole"] = -45
    ds.Polar_Stereographic.attrs["proj_params"] = "epsg:3413"
    
    return ds


In [ ]:
ds = create_circular()
Vx = np.squeeze(ds["vx"].to_numpy())
Vy = np.squeeze(ds["vy"].to_numpy())
V = np.squeeze(ds["v"].to_numpy())
x = ds["x"].to_numpy()
y = ds["y"].to_numpy()
nx = len(x)
ny = len(y)

In [ ]:
points = [Point(0, -25000), Point(0, -50000), Point(0, -75000)]

dts = [0.01, 0.1, 1, 10]
dt_trajs = []

progress = tqdm(dts, total=len(dts), leave=False, position=0)
for dt in progress:
    progress.set_description(f"Time step {dt}")
    total_times = []
    for p in points:
        r = p.distance(Point(0, 0))
        circ = 2 * r * np.pi
        vx, vy = point_velocity(Vx, Vy, x, y, p)
        v = np.sqrt(vx**2 + vy**2)
        total_time = circ / v
        total_times.append(total_time)
    
    with tqdm_joblib(tqdm(desc="Processing trajectory", total=len(points), leave=False, position=1)) as progress_bar:
        trajs = Parallel(n_jobs=1)(
            delayed(compute_trajectory)(points[k], Vx, Vy, x, y, total_time=total_times[k], dt=dt, reverse=True)
            for k in range(len(points)))
    dt_trajs.append(trajs)


In [ ]:

fig, axs = plt.subplots(1, len(dt_trajs), sharey=True, figsize=(12, 3))
ax.set_aspect("equal")
for k, trajs in enumerate(dt_trajs):
    if k == len(dt_trajs)-1:
        add_cb = True
    else:
        add_cb = False
    ds["v"].plot(ax=axs[k], vmin=0, vmax=500, add_colorbar=add_cb)
    for traj in trajs:
        ps = np.array([[p.x, p.y] for p in traj]).T
        axs[k].scatter(*ps, s=0.5)
    axs[k].set_title(f"""dt={dts[k]}""")

fig, ax = plt.subplots(1, 1, figsize=(3.2, 3.2))
for k in range(len(points)):
    ax.plot(dts, [dt_trajs[n][k][0].distance(dt_trajs[n][k][-1]) for n in range(len(dt_trajs))])
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_ylabel("distance (m)")
ax.set_xlabel("dt (years)")

In [ ]:
from pismragis.interpolation import (
    interpolate_rkf,
    velocity_at_point,
    interpolate_rkf_fast,
    velocity_at_point_fast,
)


def compute_trajectory(
    point: Point,
    Vx: Union[ndarray, DataArray],
    Vy: Union[ndarray, DataArray],
    x: Union[ndarray, DataArray],
    y: Union[ndarray, DataArray],
    dt: float = 0.1,
    total_time: float = 1000,
    reverse: bool = False,
) -> Tuple[list[Point], list]:
    """
    Compute trajectory
    """
    if reverse:
        Vx = -Vx
        Vy = -Vy
    pts = [point]
    pts_error_estim = [0.0]
    time = 0.0
    while abs(time) <= (total_time):
        point, point_error_estim = interpolate_rkf(Vx, Vy, x, y, point, delta_time=dt)
        if (point is None) or (point_error_estim is None):
            break
        pts.append(point)
        pts_error_estim.append(point_error_estim)
        time += dt
    return pts, pts_error_estim

def compute_trajectory_fast(
    point: Point,
    Vx: Union[ndarray, DataArray],
    Vy: Union[ndarray, DataArray],
    x: Union[ndarray, DataArray],
    y: Union[ndarray, DataArray],
    dt: float = 0.1,
    total_time: float = 1000,
    reverse: bool = False,
) -> Tuple[list[Point], list]:
    """
    Compute trajectory
    """
    if reverse:
        Vx = -Vx
        Vy = -Vy
    pts = [point]
    pts_error_estim = [0.0]
    time = 0.0
    while abs(time) <= (total_time):
        point, point_error_estim = interpolate_rkf_fast(Vx, Vy, x, y, point, delta_time=dt)
        if (point is None) or (point_error_estim is None):
            break
        pts.append(point)
        pts_error_estim.append(point_error_estim)
        time += dt
    return pts, pts_error_estim


In [ ]:
def old():
    for p in points:
        compute_trajectory(p, Vx, Vy, x, y)


In [ ]:
def new():
    
    for p in points:
        compute_trajectory_fast(p, Vx, Vy, x, y)


In [ ]:
from typing import Optional, Tuple, Union

import numpy as np
import scipy
from numpy import ndarray
from shapely import Point
from xarray import DataArray
from pismragis.interpolation import InterpolationMatrix

In [ ]:
p = Point(0, -50000)

In [ ]:
%timeit compute_trajectory(p, Vx, Vy, x, y)

In [ ]:
%timeit compute_trajectory_fast(p, Vx, Vy, x, y)

In [ ]:
compute_trajectory_fast(p, Vx, Vy, x, y)

## Create doc-string examples

In [ ]:
import numpy as np
from shapely import Point

nx = 201
ny = 401
x = np.linspace(-100e3, 100e3, nx)
y = np.linspace(-100e3, 100e3, ny)
X, Y = np.meshgrid(x, y)

# Directional vectors
vx = -Y / np.sqrt(X**2 + Y**2) * 250
vy = X / np.sqrt(X**2 + Y**2) * 250

p = Point(0, -50000)

pts, pts_error_estim = compute_trajectory(p, vx, vx, x, y, dt=1, total_time=10)

In [ ]:
pts